In [1]:
import time
from tqdm import tqdm
import requests
import pandas as pd
import json

errors = []


In [10]:
with open('../config.json', 'r') as file:
    config = json.load(file)

In [ ]:

collected_data = []
errors = []

def api_call(id):
    while True:
        response = requests.get(f"{config['url']}{id}")
        if response.status_code == 429:  # Rate limit hit
            time.sleep(1)
            print('429', id)
        elif response.status_code == 200:
            collected_data.append(response.json())
            return response.json()
        elif response.status_code == 504:
            print(f"504: {id}")
        else:
            errors.append({'code': response.status_code, 'id': id})
            break

def process_range(start_index, end_index, rate_limit):
    request_count = 0
    start_time = time.time()

    for id in tqdm(range(start_index, end_index)):
        api_call(id)
        request_count += 1

        if request_count >= rate_limit:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            request_count = 0
            start_time = time.time()

def batch_process(batch_size, total_requests, rate_limit):
    start = 95001
    end = start + total_requests
    for i in range(start, end, batch_size):
        end_index = min(i + batch_size, end)
        process_range(i, end_index, rate_limit)
        save_data(i, end_index)
        collected_data.clear()

def save_data(start_index, end_index):
    df = pd.json_normalize(collected_data)
    print(f"Saving data from ID {start_index} to {end_index}")
    filename = f"season-df-4-{start_index}-{end_index}.csv.gz"
    filepath = f"{config['file_path']}{filename}"
    df.to_csv(filepath, index=False, compression='gzip')

batch_size = 1000
total_requests = 4999  
rate_limit = 300  

batch_process(batch_size, total_requests, rate_limit)
